In [15]:
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten, Conv1D, pooling
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.callbacks import EarlyStopping
import pandas as pd, numpy as np
from sklearn.preprocessing import RobustScaler
import random
from numpy import *

np.random.seed(30)

test1=pd.read_csv('normal4.csv', header=None)
test2=pd.read_csv('emer4.csv', header=None)

#test1=pd.read_csv('normal4.csv')
#test2=pd.read_csv('emer4.csv')

#test1 = test1.drop(["Time", "Unnamed: 0"], axis=1)
#test2 = test2.drop(["Time", "Unnamed: 0"], axis=1)
test1.drop(test1.columns[[0]], axis='columns')
test2.drop(test1.columns[[0]], axis='columns')


#test1.head()
#test2.head()

,1,2,3,4,5,6,7,8,9
0,0.208769,0.321892,0.402616,0.822797,0.401234,0.326537,0.185503,0.525253,0.302083
1,0.020361,0.000722,0.065042,0.288284,0.115960,0.113156,0.070587,0.525253,0.302083
2,0.010590,0.020703,0.009229,0.052797,0.013179,0.020738,0.012441,0.606061,0.197917
3,0.000000,0.000000,0.003993,0.014712,0.017271,0.004819,0.007694,0.606061,0.197917
4,0.038906,0.030418,0.139989,0.573819,0.246430,0.088885,0.121300,0.606061,0.197917
...,...,...,...,...,...,...,...,...,...
250,0.004956,0.004446,0.001509,0.012464,0.004290,0.002813,0.001080,0.333333,0.385417
251,0.012144,0.011523,0.005930,0.011359,0.002607,0.009115,0.003831,0.252525,0.104167
252,0.093970,0.121758,0.143926,0.182073,0.155981,0.057003,0.029367,0.252525,0.104167
253,0.001399,0.008065,0.003465,0.013080,0.016776,0.003162,0.002390,0.252525,0.104167


In [21]:
test1["label"] = "unsafe"
test2["label"] = "safe"

X=pd.concat([test1, test2])

#print(X.shape)
#print(X)
#"""
#Y를 범주형 데이터로 전환
bclass = {'safe':[1, 0], 'unsafe':[0, 1]}
y = np.empty((1123, 2))
for i, v in enumerate(X['label']):
    y[i] = bclass[v]

#X축의 label을 제외한 테스트 데이터 셋
del X['label']
#X = X[['delta','theta','lowAlpha','highAlpha','lowBeta','highBeta','lowGamma','midGamma']].to_numpy()
X=X.to_numpy()
X.shape
#"""

(1123, 10)

In [22]:
tmp = [[x,y_tmp] for x, y_tmp in zip(X, y)]
random.shuffle(tmp)
X= [n[0] for n in tmp]
y = [n[1] for n in tmp]

X = np.asarray(X, dtype=np.float32)
y = np.asarray(y, dtype=np.float32)

#print(X.shape, y.shape)

(1123, 10) (1123, 2)


In [24]:
#scaler = RobustScaler()
#scaler.fit(X)
#X = scaler.transform(X)
#print(X)
#np.mean(X), np.std(X)

(0.18474834, 0.22287035)

In [25]:
x_train, y_train = X[1:900], y[1:900]
x_test, y_test = X[900:], y[900:]

In [26]:
x_train = x_train.reshape(x_train.shape[0], 10, 1)
x_test = x_test.reshape(x_test.shape[0], 10, 1)

print('x_train shape:', x_train.shape[1])
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print(y_train.shape[1])

x_train shape: 10
899 train samples
223 test samples
2


In [27]:
def evaluate_model(x_train, y_train, x_test, y_test):
    model = Sequential()
    verbose, epochs, batch_size = 0, 10, 32
    n_timesteps, n_features, n_outputs = x_train.shape[1], x_train.shape[2], y_train.shape[1]
    model.add(Convolution1D(filters=64, kernel_size=3, activation='relu',
                        input_shape=(n_timesteps,n_features)))
    model.add(Convolution1D(filters=64, kernel_size = 3, activation='relu'))
    model.add(Dropout(0.25))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_outputs, activation='softmax'))

    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    hist=model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
    print(hist.history['loss'])

    _, accuracy= model.evaluate(x_test, y_test, batch_size=batch_size, verbose=0)
    return accuracy

In [28]:
repeats=10
scores = list()
for r in range(repeats):
	score = evaluate_model(x_train, y_train, x_test, y_test)
	score = score * 100.0
	print('>#%d: %.3f' % (r+1, score))
	scores.append(score)
    
# summarize results
print(scores)
m, s = mean(scores), std(scores)
print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

[0.5831472284271402, 0.5244607009134515, 0.5035074741079226, 0.48598878843102755, 0.45625836596937147, 0.4380480813502735, 0.42045927299673486, 0.3972457411954878, 0.39904908211000506, 0.38431062316470205]
>#1: 88.341
[0.5637404582524326, 0.5198317754918397, 0.484724100525838, 0.46027572988402987, 0.4222063119207261, 0.40993935153203226, 0.38138224338663035, 0.38813106857812046, 0.36001170709215363, 0.3611691725399921]
>#2: 86.996
[0.596971854899033, 0.5272373915116434, 0.492469535355308, 0.4547854667908623, 0.4275492368073299, 0.41074250577156485, 0.39183861796678904, 0.3965509492112479, 0.3723635203448498, 0.3496353055067667]
>#3: 87.892
[0.5863525683145236, 0.5191968334662636, 0.4820307640398172, 0.45367434630802395, 0.43942322987271626, 0.40597905041245913, 0.4054122054628257, 0.38499221809050926, 0.37021432496250933, 0.37221292420409546]
>#4: 87.444
[0.5845297515989013, 0.5301408008553162, 0.5021550380547134, 0.46299622215338887, 0.43081656446446304, 0.4365779852641703, 0.40591897